In [6]:
#Red Neuronal Convolucional Basica
!pip show tensorflow_datasets



/bin/bash: /home/albertocastrovill/Documents/UDEM/IA2/IA2_venv/bin/pip: /home/albertocastrovill/Documents/UDEM/IA2_venv/bin/python3: bad interpreter: No such file or directory


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds